![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_DistilBertForSequenceClassification.ipynb)

## Import ONNX DistlBertForSequenceClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `DistlBertForSequenceClassification` is only available since in `Spark NLP 5.1.3` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import DistilBERT models trained/fine-tuned for token classification via `DistilBertForSequenceClassification` or `TFDistilBertForSequenceClassification`. These models are usually under `Token Classification` category and have `bert` in their labels
- Reference: [TFDistilBertForSequenceClassification](https://huggingface.co/transformers/model_doc/distilbert.html#tfdistilbertforsequenceclassification)
- Some [example models](https://huggingface.co/models?filter=distilbert&pipeline_tag=text-classification)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.48.3 optimum onnx

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [distilbert-base-uncased-finetuned-sst-2-english](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)  model from HuggingFace as an example and load it as a `ORTModelForSequenceClassification`, representing an ONNX model.
- In addition to the DistilBERT model, we also need to save the `DistilBertTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import DistilBertTokenizer
from optimum.onnxruntime import ORTModelForSequenceClassification

MODEL_NAME = 'distilbert-base-uncased-finetuned-sst-2-english'
ONNX_MODEL = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForSequenceClassification.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(ONNX_MODEL)

tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(ONNX_MODEL)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The model distilbert-base-uncased-finetuned-sst-2-english was already converted to ONNX but got `export=True`, the model will be converted to ONNX once again. Don't forget to save the resulting model with `.save_pretrained()`


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

('onnx_models/distilbert-base-uncased-finetuned-sst-2-english/tokenizer_config.json',
 'onnx_models/distilbert-base-uncased-finetuned-sst-2-english/special_tokens_map.json',
 'onnx_models/distilbert-base-uncased-finetuned-sst-2-english/vocab.txt',
 'onnx_models/distilbert-base-uncased-finetuned-sst-2-english/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {ONNX_MODEL}

total 261924
-rw-r--r-- 1 root root       735 Jun 14 00:09 config.json
-rw-r--r-- 1 root root 267958137 Jun 14 00:09 model.onnx
-rw-r--r-- 1 root root       125 Jun 14 00:09 special_tokens_map.json
-rw-r--r-- 1 root root      1278 Jun 14 00:09 tokenizer_config.json
-rw-r--r-- 1 root root    231508 Jun 14 00:09 vocab.txt


- As you can see, we need to move `vocab.txt` to assets folder which Spark NLP will look for
- We also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [4]:
!mkdir -p {ONNX_MODEL}/assets

labels = ort_model.config.id2label
labels = [label for _, label in sorted(labels.items())]

with open(f"{ONNX_MODEL}/assets/labels.txt", "w") as f:
    f.write("\n".join(labels))

!mv {ONNX_MODEL}/vocab.txt {ONNX_MODEL}/assets/

In [5]:
!ls -lR {ONNX_MODEL}

onnx_models/distilbert-base-uncased-finetuned-sst-2-english:
total 261700
drwxr-xr-x 2 root root      4096 Jun 14 00:09 assets
-rw-r--r-- 1 root root       735 Jun 14 00:09 config.json
-rw-r--r-- 1 root root 267958137 Jun 14 00:09 model.onnx
-rw-r--r-- 1 root root       125 Jun 14 00:09 special_tokens_map.json
-rw-r--r-- 1 root root      1278 Jun 14 00:09 tokenizer_config.json

onnx_models/distilbert-base-uncased-finetuned-sst-2-english/assets:
total 232
-rw-r--r-- 1 root root     17 Jun 14 00:09 labels.txt
-rw-r--r-- 1 root root 231508 Jun 14 00:09 vocab.txt


Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

## Import and Save DistilBertForSequenceClassification in Spark NLP


- **Install and set up Spark NLP in Google Colab**
  - This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.


In [6]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 36.5 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `DistilBertForSequenceClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DistilBertForSequenceClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [8]:
from sparknlp.annotator import DistilBertForSequenceClassification

sequenceClassifier = DistilBertForSequenceClassification.loadSavedModel(
      ONNX_MODEL,
      spark
      )\
    .setInputCols(["document",'token'])\
    .setOutputCol("class")\
    .setCaseSensitive(True)\
    .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
sequenceClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {ONNX_MODEL}

Awesome 😎  !

This is your AlbertForSequenceClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [ ]:
! ls -l {ONNX_MODEL}_spark_nlp_onnx

Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny AlbertForSequenceClassification model 😊

In [12]:
sequenceClassifier_loaded = DistilBertForSequenceClassification.load("./{}_spark_nlp_onnx".format(ONNX_MODEL))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

You can see what labels were used to train this model via `getClasses` function:

In [13]:
sequenceClassifier_loaded.getClasses()

['POSITIVE', 'NEGATIVE']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [14]:
from pyspark.ml import Pipeline
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer

document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    sequenceClassifier_loaded
])

example = spark.createDataFrame([
    ["I love you!"],
    ["I feel lucky to be here."],
    ["I hate her!"]
]).toDF("text")

result = pipeline.fit(example).transform(example)
result.select("text", "class.result").show(truncate=False)

+------------------------+----------+
|text                    |result    |
+------------------------+----------+
|I love you!             |[POSITIVE]|
|I feel lucky to be here.|[POSITIVE]|
|I hate her!             |[NEGATIVE]|
+------------------------+----------+



That's it! You can now go wild and use hundreds of `DistlBertForSequenceClassification` models from HuggingFace 🤗 in Spark NLP 🚀
